In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

# For metrics and splitting
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import ParameterGrid

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from dl_models_lib import sliding_window_forecast_with_torch
from dl_models_lib import MLP, CNN, RNN, LSTM, GRU

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [5]:
def tune_torch_model(
    df,
    target_col,
    model_type,
    param_grid,
    window_sizes=[25, 50],
    test_ratio=0.2,
    drop_cols=None,
    log_dir="model_dl_logs"
):
    """
    Iterates over all parameter combos in `param_grid` for the given `model_type`,
    and for each window_size in `window_sizes`, calls sliding_window_forecast_with_torch.
    Logs metrics and returns a summary DataFrame.
    """
    if drop_cols is None:
        drop_cols = []
    
    param_list = list(ParameterGrid(param_grid))
    total_iterations = len(param_list) * len(window_sizes)
    pbar = tqdm(total=total_iterations, desc=f"Tuning {model_type}")

    summaries = []
    for w in window_sizes:
        for params in param_list:
            log_filename = f"{model_type}_window_{w}.csv"

            log_df = sliding_window_forecast_with_torch(
                df=df,
                target_col=target_col,
                model_name=model_type,
                model_params=params,
                window_size=w,
                test_ratio=test_ratio,
                drop_cols=drop_cols,
                log_dir=log_dir,
                log_filename=log_filename
            )
            
            pbar.update(1)
            if log_df.empty:
                continue

            avg_mse  = log_df["MSE_score"].mean()
            avg_mape = log_df["MAPE_score"].mean()
            avg_r2   = log_df["R^2_score"].mean()

            summary_entry = {
                "model_name": model_type,
                "model_hyperparameters_dict": json.dumps(params),
                "window_size": w,
                "test_ratio": test_ratio,
                "avg_MSE": avg_mse,
                "avg_MAPE": avg_mape,
                "avg_R^2": avg_r2
            }
            summaries.append(summary_entry)
    pbar.close()
    
    return pd.DataFrame(summaries) if summaries else pd.DataFrame()

In [6]:
def combine_and_top_logs(log_dir="model_dl_logs", tops=5):
    """
    Reads all CSV files in the given log directory, concatenates them,
    groups by (model_name, model_hyperparameters_dict),
    calculates mean MSE, MAPE, R^2, and returns the top `tops` per model by MAPE ascending.
    """
    all_files = [f for f in os.listdir(log_dir) if f.endswith(".csv")]
    if not all_files:
        print(f"No CSV log files in {log_dir} directory.")
        return pd.DataFrame()
    
    dfs = []
    for csvf in all_files:
        path = os.path.join(log_dir, csvf)
        tmp_df = pd.read_csv(path)
        dfs.append(tmp_df)
    combined_logs = pd.concat(dfs, ignore_index=True)
    
    grouped_summary = combined_logs.groupby(
        ["model_name","model_hyperparameters_dict"]
    )[["MSE_score","MAPE_score","R^2_score"]].mean().reset_index()

    top_n_list = []
    for model in grouped_summary["model_name"].unique():
        sub = grouped_summary[grouped_summary["model_name"] == model]
        top_n = sub.sort_values("MAPE_score", ascending=True).head(tops)
        top_n_list.append(top_n)
    top_n_combined = pd.concat(top_n_list, ignore_index=True)
    return top_n_combined


In [7]:
param_grids_torch = {
    "MLP": {
        "hidden_dim":   [32, 64],
        "n_layers":     [1, 2],
        "dropout":      [0.0, 0.2],
        "learning_rate":[1e-2, 1e-3],
        "optimizer":    ["adam", "sgd"],
        "momentum":     [0.0, 0.9],  # only used if optimizer=sgd
        "epochs":       [25, 50],
        "batch_size":   [16, 32],
        "weight_decay": [0.0, 1e-4],
        "activation":   ["relu", "tanh"],
        "use_batchnorm":[False, True],
        "scheduler":    ["none", "step"]
    },

    "CNN": {
        "hidden_dim":   [32, 64],
        "n_layers":     [1, 2],
        "kernel_size":  [3],
        "pooling_type": ["max", "avg"],
        "dropout":      [0.0, 0.2],
        "learning_rate":[1e-2, 1e-3],
        "optimizer":    ["adam", "sgd"],
        "momentum":     [0.0, 0.9],
        "epochs":       [25, 50],
        "batch_size":   [16, 32],
        "weight_decay": [0.0, 1e-4],
        "scheduler":    ["none", "step"]
    },

    "RNN": {
        "hidden_dim":   [32, 64],
        "n_layers":     [1, 2],
        "dropout":      [0.0, 0.2],
        "learning_rate":[1e-2, 1e-3],
        "optimizer":    ["adam", "sgd"],
        "momentum":     [0.0, 0.9],
        "epochs":       [25, 50],
        "batch_size":   [16, 32],
        "weight_decay": [0.0, 1e-4],
        "bidirectional":[True, False],
        "scheduler":    ["none", "step"]
    },

    "LSTM": {
        "hidden_dim":   [32, 64],
        "n_layers":     [1, 2],
        "dropout":      [0.0, 0.2],
        "learning_rate":[1e-2, 1e-3],
        "optimizer":    ["adam", "sgd"],
        "momentum":     [0.0, 0.9],
        "epochs":       [25, 50],
        "batch_size":   [16, 32],
        "weight_decay": [0.0, 1e-4],
        "bidirectional":[True, False],
        "recurrent_dropout":[0.0],  # Not really used unless you do custom LSTM cell
        "scheduler":    ["none", "step"]
    },

    "GRU": {
        "hidden_dim":   [32, 64],
        "n_layers":     [1, 2],
        "dropout":      [0.0, 0.2],
        "learning_rate":[1e-2, 1e-3],
        "optimizer":    ["adam", "sgd"],
        "momentum":     [0.0, 0.9],
        "epochs":       [25, 50],
        "batch_size":   [16, 32],
        "weight_decay": [0.0, 1e-4],
        "bidirectional":[True, False],
        "recurrent_dropout":[0.0],
        "scheduler":    ["none", "step"]
    }
}

In [8]:
with open("temp_output/quarterly_X_y.pkl", "rb") as f:
    data_q = pickle.load(f)

drop_columns = []
print(data_q.keys())  # Should show all the tickers like AAPL, MSFT, etc.
quaterly_data = data_q["AAPL"]

dict_keys(['AAPL', 'MSFT', 'LLY', 'UNH', 'V', 'MA', 'GOOGL', 'META', 'AMZN', 'TSLA', 'PG', 'WMT', 'RTX', 'UNP', 'XOM', 'CVX', 'LIN', 'SHW', 'AMT', 'PLD', 'NEE', 'SO'])


In [ ]:
# MLP on quarterly
mlp_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="MLP",
    param_grid=param_grids_torch["MLP"],
    window_sizes=[10],  # or [25,50]
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

# CNN on quarterly
cnn_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="CNN",
    param_grid=param_grids_torch["CNN"],
    window_sizes=[10],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

# RNN on quarterly
rnn_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="RNN",
    param_grid=param_grids_torch["RNN"],
    window_sizes=[10],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

# LSTM on quarterly
lstm_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="LSTM",
    param_grid=param_grids_torch["LSTM"],
    window_sizes=[10],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

# GRU on quarterly
gru_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="GRU",
    param_grid=param_grids_torch["GRU"],
    window_sizes=[10],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

Tuning MLP:  35%|███▍      | 1429/4096 [1:08:51<3:09:16,  4.26s/it]

In [ ]:
combined_summary_q = pd.concat([mlp_summary_q, cnn_summary_q, rnn_summary_q, lstm_summary_q, gru_summary_q], ignore_index=True)
print("Combined Tuning Summary:")
display(combined_summary_q)

# Alternatively, read all prediction log files from log directory
top5_summary_q = combine_and_top_logs(log_dir="model_dl_logs_q")
print("Top 5 Configurations per Model:")
display(top5_summary_q)

Combined Tuning Summary:


,model_name,model_hyperparameters_dict,window_size,test_ratio,avg_MSE,avg_MAPE,avg_R^2
0,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,778.982207,0.177793,-183.826692
1,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,769.949728,0.176999,-200.679550
2,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,835.726052,0.181807,-198.160551
3,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,798.616589,0.180312,-177.330999
4,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,787.044281,0.179981,-173.335006
...,...,...,...,...,...,...,...
8743,GRU,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 1...",10,0.2,816.831696,0.183870,-220.677119
8744,GRU,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 1...",10,0.2,816.741247,0.183856,-220.627070
8745,GRU,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 1...",10,0.2,816.653085,0.183842,-220.581051
8746,GRU,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 1...",10,0.2,816.567877,0.183829,-220.532425


Top 5 Configurations per Model:


,model_name,model_hyperparameters_dict,MSE_score,MAPE_score,R^2_score
0,GRU,"{""batch_size"": 16, ""dropout"": 0.4, ""epochs"": 2...",604.429245,0.148500,-114.641782
1,GRU,"{""batch_size"": 32, ""dropout"": 0.4, ""epochs"": 2...",624.912527,0.149950,-118.385994
2,GRU,"{""batch_size"": 16, ""dropout"": 0.4, ""epochs"": 2...",638.081727,0.150358,-134.695980
3,GRU,"{""batch_size"": 16, ""dropout"": 0.4, ""epochs"": 1...",604.974328,0.150906,-120.426897
4,GRU,"{""batch_size"": 32, ""dropout"": 0.4, ""epochs"": 1...",599.377000,0.150942,-108.714437
5,LSTM,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",592.196254,0.145152,-73.316614
6,LSTM,"{""batch_size"": 32, ""dropout"": 0.4, ""epochs"": 2...",603.333849,0.147617,-169.880725
7,LSTM,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",592.335330,0.148016,-82.748094
8,LSTM,"{""batch_size"": 16, ""dropout"": 0.4, ""epochs"": 2...",602.140783,0.148135,-153.402797
9,LSTM,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 2...",589.238387,0.148434,-160.610464


In [ ]:
with open("temp_output/daily_X_y.pkl", "rb") as f:
    data_d = pickle.load(f)
    
drop_columns = []
print(data_d.keys())  # Should show all the tickers like AAPL, MSFT, etc.
daily_data = data_d["AAPL"]
daily_data = daily_data.iloc[-128:,:]
daily_data

dict_keys(['AAPL', 'MSFT', 'LLY', 'UNH', 'V', 'MA', 'GOOGL', 'META', 'AMZN', 'TSLA', 'PG', 'WMT', 'RTX', 'UNP', 'XOM', 'CVX', 'LIN', 'SHW', 'AMT', 'PLD', 'NEE', 'SO', '^GSPC'])


,Adj Close,Close,High,Low,Open,Volume,SMA5,SMA50,SMA200,MACDLine,...,^IXIC,^DJI,^VIX,CL=F,GC=F,SI=F,^TNX,DX-Y.NYB,FedFundsRate,y
2024-07-01,216.023956,216.750000,217.509995,211.919998,212.089996,60402900,212.045334,190.963962,182.925126,6.332360,...,17879.300781,39169.519531,12.22,83.379997,2327.600098,29.299999,4.479,105.900002,5.33,219.532166
2024-07-02,219.532166,220.270004,220.380005,215.100006,216.149994,58046200,214.277829,192.070114,183.150480,6.644230,...,18028.759766,39331.851562,12.03,82.809998,2323.000000,29.353001,4.436,105.720001,5.33,220.807892
2024-07-03,220.807892,221.550003,221.550003,219.029999,220.000000,37369800,215.932269,193.185060,183.385837,6.914623,...,18188.300781,39308.000000,12.09,83.879997,2359.800049,30.548000,4.355,105.400002,5.33,225.581833
2024-07-05,225.581833,226.339996,226.449997,221.649994,221.649994,60412400,218.372064,194.374384,183.630370,7.428497,...,18352.759766,39375.871094,12.48,83.160004,2388.500000,31.388000,4.272,104.879997,5.33,227.056885
2024-07-08,227.056885,227.820007,227.850006,223.250000,227.089996,59085900,221.800546,195.551008,183.876819,7.864117,...,18403.740234,39344.789062,12.37,82.330002,2355.199951,30.618000,4.269,105.010002,5.33,227.913986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,257.916443,258.200012,258.209991,255.289993,255.490005,23234700,252.881967,235.564016,211.182570,6.067327,...,20031.130859,43297.031250,14.27,70.099998,2620.000000,29.974001,4.591,108.260002,4.33,258.735504
2024-12-26,258.735504,259.019989,260.100006,257.630005,258.190002,27237100,255.073553,236.071997,211.614168,6.300019,...,20020.359375,43325.800781,14.73,69.620003,2638.800049,30.047001,4.579,108.129997,4.33,255.309296
2024-12-27,255.309296,255.589996,258.700012,253.059998,257.829987,42355300,256.232281,236.552763,212.039086,6.137217,...,19722.029297,42992.210938,15.95,70.599998,2617.199951,29.655001,4.619,108.000000,4.33,251.923019
2024-12-30,251.923019,252.199997,253.500000,250.750000,252.229996,35557500,255.774783,236.958419,212.437766,5.669595,...,19486.789062,42573.730469,17.40,70.989998,2606.100098,29.106001,4.545,108.129997,4.33,250.144974


In [ ]:
# MLP
mlp_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="MLP",
    param_grid=param_grids_torch["MLP"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

# CNN on quarterly
cnn_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="CNN",
    param_grid=param_grids_torch["CNN"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

# RNN
rnn_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="RNN",
    param_grid=param_grids_torch["RNN"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

# LSTM
lstm_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="LSTM",
    param_grid=param_grids_torch["LSTM"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

# GRU
gru_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="GRU",
    param_grid=param_grids_torch["GRU"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

Tuning MLP:  19%|█▊        | 410/2187 [2:00:23<15:03:59, 30.52s/it]

In [ ]:
combined_summary_d = pd.concat([mlp_summary_d, cnn_summary_d, rnn_summary_d, lstm_summary_d, gru_summary_d], ignore_index=True)
print("Combined Tuning Summary:")
display(combined_summary_d)

# Alternatively, read all prediction log files from log directory
top5_summary_d = combine_and_top_logs(log_dir="model_dl_logs_d")
print("Top 5 Configurations per Model:")
display(top5_summary_d)

In [ ]:
top_summary_q = combine_and_top_logs(log_dir="model_dl_logs_q", tops=25)
for d in top_summary_q['model_hyperparameters_dict']:
    print(json.loads(d))
    
print("\n"+"="*100+"\n")

top_summary_d = combine_and_top_logs(log_dir="model_dl_logs_d", tops=25)
for d in top_summary_d['model_hyperparameters_dict']:
    print(json.loads(d))

{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 8000}
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 6400}
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 4800}
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 3200}
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 1600}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 8000}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 6400}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 4800}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 3200}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 1600}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 4800}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 6400}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 8000}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 3200}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 1600}
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 1600}
{'learni